# Pylon Lottery - Galactic Punk

### Pylon Lottery
The pylon lottery is a mechanism implemented in the [Pylon](https://pylon.money/) protocol, in which users can deposit UST in exchange for lottey tickets. 

Each 10 UST deposited is rewarded with 1 ticket. The longer the UST remain in the pool, the more tickets the user is rewarded. At the end of the lottey period, a winning ticket is extracted from the ones rewarded to users and the lucky user owning the winning ticket wins the price. All the deposited UST are then returned to the depositors.

In this analysis we focus on the Pylon lottery which has Galactic Punk NFTs as rewards.

### Galactic Punks
I quote from the pylon [protocol page](https://gateway.pylon.money/nft/galactic-punks): "*Galactic Punks are 10,921 randomly generated NFTs on the Terra blockchain. Attributes metadata will be revealed alongside rarity rankings. 10,921 unique characters generated from 7 categories of traits. Around 80 unique traits possible on each punk ranging from common to ultra rare.*"


### Objective
In this notebook we will explore the Terra blockchain data to obtain insights in the participation in the Lottery. In specific, we will try to answer the two following questions:
* How much UST has been deposited into the lottery since it’s inception?
* What has been the average UST deposit amount?

### Data
The data has been kindly provided by [Flipside](https://app.flipsidecrypto.com/velocity) and the query used to retrieve the relevant data can be found [here](https://app.flipsidecrypto.com/velocity/queries/5027ee2e-4de7-41dd-8686-072046b3ce8b).
* **Timeframe** The analysis has been performed on the 7th November 2021, and therefore the latest update of the data used is dated on the that day. However, as we can see in the data itself, the lottery first opened on the 27th of October and therefore the earliest data is dated accordingly.
* **Data structure** each line represents one interaction with the GP pylon lottery smart contract. One interaction can either represent one deposit or one withdrawal
* **Formatting** The amounts of UST is represented in uUST and we therefore divide the numbers by 1000000 to obtain the value in UST

### Query
The query used to retrieve the relevant data can be found [here](https://app.flipsidecrypto.com/velocity/queries/5027ee2e-4de7-41dd-8686-072046b3ce8b). Some main points to highlight:
* Only the successful transactions have been filtered
* Only the transactions interacting with the GP pylon lottery smart contract (**terra126zhrrpkckjs82elgutz5qmqa4zjndfwk5dr2t**) have been filtered
* In order to understand which operation has been performed in the transaction (deposit or withdrawal) we have investigated the transactions in the Terra explorer and identified the field **contract** in the execute_mgs has the value (**terra10jrv8wy6s06mku9t6yawt2yr09wjlqsw0qk0vf**) in case of deposits and (**terra1jk0xh49ft2ls4u9dlfqweed8080u6ysumvmtcz**) withdrawals. We labeled the transactions accordingly. 

## Analysis

**Imports**

In [232]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (15.0, 5.0)
pd.set_option('display.max_columns', 120)
pd.options.display.max_colwidth = 10000

In [233]:
pylon_gp_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/5027ee2e-4de7-41dd-8686-072046b3ce8b/data/latest')

In [234]:
pylon_gp_df.head(2)

,BLOCK_TIMESTAMP,MSG_VALUE:COINS,MSG_VALUE:CONTRACT,MSG_VALUE:EXECUTE_MSG,MSG_VALUE:EXECUTE_MSG:SEND:AMOUNT,MSG_VALUE:EXECUTE_MSG:SEND:DENOM,MSG_VALUE:SENDER,TX_ID,TYPE_OP
0,2021-11-06T15:22:08Z,[],"""terra126zhrrpkckjs82elgutz5qmqa4zjndfwk5dr2t""","{\n ""send"": {\n ""amount"": ""300006513"",\n ""contract"": ""terra1jk0xh49ft2ls4u9dlfqweed8080u6ysumvmtcz"",\n ""msg"": ""eyJyZWRlZW0iOnt9fQ==""\n }\n}","""300006513""",NaN,"""terra18rwg4ljlsjdcpjv82luas7pnq6jtjjcu0jqwm5""",86ABF3421E3685C0A9A37E8F2B164BB95D3CA1FBAF351D2BA57CC383ABE2A81C,WITHDRAW
1,2021-11-06T10:46:05Z,[],"""terra126zhrrpkckjs82elgutz5qmqa4zjndfwk5dr2t""","{\n ""send"": {\n ""amount"": ""250005091"",\n ""contract"": ""terra10jrv8wy6s06mku9t6yawt2yr09wjlqsw0qk0vf"",\n ""msg"": ""eyJkZXBvc2l0Ijp7fX0=""\n }\n}","""250005091""",NaN,"""terra16dn5f60uwfqaly6k4geckwr4xswya6ctw2ana6""",9A5D1E71037E0203120400E697739F8CFD003F9908DF5B8DC2799C7DACB18D4A,DEPOSIT


**Data cleaning and preparation**

In [235]:
pylon_gp_df.columns = ["block_timestamp","coins","contract","execute_msg","send_amount","send_denom","sender","tx_id","type_op"]
pylon_gp_df=pylon_gp_df.drop("coins", axis=1)
pylon_gp_df.send_amount=pylon_gp_df.send_amount.apply(lambda x: x.replace("\"","")).fillna(0).astype('float64')
pylon_gp_df.sender=pylon_gp_df.sender.apply(lambda x: x.replace("\"",""))
pylon_gp_df.contract=pylon_gp_df.contract.apply(lambda x: x.replace("\"",""))
pylon_gp_df.block_timestamp = pd.to_datetime(pylon_gp_df.block_timestamp)
pylon_gp_df["month_day"] = pylon_gp_df.block_timestamp.apply(lambda t: f"{t.month}-{t.day}")
pylon_gp_df = pylon_gp_df.sort_values(by='block_timestamp')

In [236]:
pylon_gp_df.head(2)

,block_timestamp,contract,execute_msg,send_amount,send_denom,sender,tx_id,type_op,month_day
1412,2021-10-27 13:00:05+00:00,terra126zhrrpkckjs82elgutz5qmqa4zjndfwk5dr2t,"{\n ""send"": {\n ""amount"": ""99997274"",\n ""contract"": ""terra10jrv8wy6s06mku9t6yawt2yr09wjlqsw0qk0vf"",\n ""msg"": ""eyJkZXBvc2l0Ijp7fX0=""\n }\n}",99997274.0,NaN,terra1hzsem376yhguf406dq7qj0a203a6y2527zmkpd,1F4726146EA56E85A89712048235D6146297C3EBFE3D72814E1A68D70554FE54,DEPOSIT,10-27
1413,2021-10-27 13:00:05+00:00,terra126zhrrpkckjs82elgutz5qmqa4zjndfwk5dr2t,"{\n ""send"": {\n ""amount"": ""198994576"",\n ""contract"": ""terra10jrv8wy6s06mku9t6yawt2yr09wjlqsw0qk0vf"",\n ""msg"": ""eyJkZXBvc2l0Ijp7fX0=""\n }\n}",198994576.0,NaN,terra1h59hphet9gvdarr58twppla0qcln5g26w9dkyh,28066F8BC0052589E5663003AA44DC44A3A8DF8181369B4B4EB38274E2B802F5,DEPOSIT,10-27


### Question 1: How much UST has been deposited into the lottery since it’s inception?

When interacting with the GP lottery smart contract, there are two operations that can be performed: deposit and withdraw.
In order to calculate the total amount of UST deposited since its inception we have to first sum the amounts of UST deposited and the subtract the sum of all the UST withdrawn.

We start by calculating the total amount deposited

In [237]:
total_deposited = pylon_gp_df[pylon_gp_df.type_op == 'DEPOSIT'].send_amount.sum()/1000000
total_deposited

9182037.559353

The total amount deposited since its inception is: 9182037 UST

Now, let's calculate the total withdrawn since its inception:

In [238]:
total_withdrawn = pylon_gp_df[pylon_gp_df.type_op == 'WITHDRAW'].send_amount.sum()/1000000
total_withdrawn

5773096.696159

The total amount deposited since its inception is: 5773096 UST.

Now we can calculate the net deposit into the lottery pool to date.

In [230]:
total_deposited - total_withdrawn

3408940.863194

The net deposit into the lottery to date is therefore: 3408940 UST

### Question 2: What has been the average UST deposit amount?

As explained earlier, there are two operations possibile. We therefore filter only the deposit operations and calculate the average amount in UST per deposit.

In [231]:
pylon_gp_df[pylon_gp_df.type_op == 'DEPOSIT'].send_amount.mean()/1000000

1976.757278655113

The average amount is therefore 1976 UST